In [1]:
import torch
from torch import nn

import cheetah

In [2]:
quadrupole = cheetah.Quadrupole(length=torch.tensor(0.2), k1=torch.tensor(4.2))
quadrupole

Quadrupole(length=tensor(0.2000), k1=tensor(4.2000), misalignment=tensor([0., 0.]), tilt=tensor(0.), name=None, device="'cpu'")

In [3]:
quadrupole.k1

tensor(4.2000)

In [4]:
list(quadrupole.parameters())

[]

In [5]:
quadrupole.k1 = nn.Parameter(torch.tensor(5.0))
quadrupole

Quadrupole(length=tensor(0.2000), k1=Parameter containing:
tensor(5., requires_grad=True), misalignment=tensor([0., 0.]), tilt=tensor(0.), name=None, device="'cpu'")

In [6]:
list(quadrupole.parameters())

[Parameter containing:
 tensor(5., requires_grad=True)]

In [7]:
segment = cheetah.Segment(elements=[quadrupole])
segment

Segment(elements=ModuleList(
  (0): Quadrupole(length=tensor(0.2000), k1=Parameter containing:
  tensor(5., requires_grad=True), misalignment=tensor([0., 0.]), tilt=tensor(0.), name=None, device="'cpu'")
), name='unnamed', device='cpu')

In [8]:
list(segment.parameters())

[Parameter containing:
 tensor(5., requires_grad=True)]

In [9]:
type(segment.elements[0].k1)

torch.nn.parameter.Parameter

In [10]:
class TestModule(nn.Module):
    """Test"""

    def __init__(self, k1: torch.Tensor = torch.tensor(0.0)) -> None:
        super().__init__()

        self.k1 = k1


test_module = TestModule(k1=torch.tensor(4.2))
list(test_module.parameters())

[]

In [11]:
test_module.k1 = nn.Parameter(torch.tensor(3.14))
list(test_module.parameters())

[Parameter containing:
 tensor(3.1400, requires_grad=True)]

In [12]:
s = "hello"
print(s)
print(repr(s))

hello
'hello'


In [13]:
segment = cheetah.Segment(
    [
        cheetah.Drift(length=torch.tensor(1.0)),
        cheetah.Quadrupole(
            length=torch.tensor(0.2),
            k1=nn.Parameter(torch.tensor(3.142)),
            name="my_quad",
        ),
        cheetah.Drift(length=torch.tensor(1.0)),
    ]
)
incoming_beam = cheetah.ParticleBeam.from_astra(
    "benchmark/astra/ACHIP_EA1_2021.1351.001"
)

outgoing_beam = segment.track(incoming_beam)

In [14]:
outgoing_beam

ParticleBeam(n=100000, mu_x=tensor(3.5298e-07, grad_fn=<MeanBackward0>), mu_xp=tensor(-4.8779e-07, grad_fn=<MeanBackward0>), mu_y=tensor(-3.2951e-06, grad_fn=<MeanBackward0>), mu_yp=tensor(-1.3088e-06, grad_fn=<MeanBackward0>), sigma_x=tensor(6.1099e-05, grad_fn=<StdBackward0>), sigma_xp=tensor(0.0001, grad_fn=<StdBackward0>), sigma_y=tensor(0.0003, grad_fn=<StdBackward0>), sigma_yp=tensor(0.0001, grad_fn=<StdBackward0>), sigma_s=tensor(8.0274e-06, grad_fn=<StdBackward0>), sigma_p=tensor(0.0023, grad_fn=<StdBackward0>), energy=tensor(1.0732e+08, dtype=torch.float64))

In [15]:
outgoing_beam.particles

tensor([[ 2.5138e-08, -2.9385e-08, -9.6960e-07,  ...,  9.0621e-21,
          2.2205e-16,  1.0000e+00],
        [ 2.0749e-05, -3.7642e-05, -9.7300e-07,  ...,  2.1451e-06,
         -1.0257e-03,  1.0000e+00],
        [ 1.6727e-08, -1.2108e-08,  7.5594e-05,  ...,  7.3348e-07,
          1.1856e-03,  1.0000e+00],
        ...,
        [ 7.5297e-05, -1.2659e-04,  3.6216e-04,  ...,  8.0915e-07,
         -5.4731e-04,  1.0000e+00],
        [ 1.9986e-05, -3.4357e-05,  4.9716e-04,  ...,  1.2477e-06,
          1.4368e-03,  1.0000e+00],
        [-2.3671e-05,  3.7790e-05,  3.8850e-04,  ...,  6.5052e-06,
         -1.8158e-03,  1.0000e+00]], grad_fn=<MmBackward0>)

In [16]:
outgoing_beam.mu_x

tensor(3.5298e-07, grad_fn=<MeanBackward0>)

In [17]:
segment.elements[1].k1

Parameter containing:
tensor(3.1420, requires_grad=True)

In [18]:
segment.elements[1].transfer_map(energy=6e6)

tensor([[ 0.9378,  0.1958,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.6153,  0.9378,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.0635,  0.2042,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.6416,  1.0635,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000, -0.0015,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000]],
       grad_fn=<CopySlices>)

In [19]:
segment.elements[1].k1 * 5

tensor(15.7100, grad_fn=<MulBackward0>)

In [20]:
torch.tensor([1.0, segment.elements[1].k1, 3.0])

tensor([1.0000, 3.1420, 3.0000])

In [21]:
torch.Tensor([1.0, segment.elements[1].k1, 3.0])

tensor([1.0000, 3.1420, 3.0000])

In [22]:
a = torch.tensor([1.0, 2.0, 3.0])
a[1] = segment.elements[1].k1
a

tensor([1.0000, 3.1420, 3.0000], grad_fn=<CopySlices>)

In [23]:
x = torch.tensor(1.0)
x.requires_grad = True
y = torch.tensor(2.0)
torch.stack([x, y])

tensor([1., 2.], grad_fn=<StackBackward0>)

In [24]:
xx = torch.tensor(1.0)
xy = torch.tensor(2.0)
yx = torch.tensor(3.0)
yy = torch.tensor(4.0)

xy.requires_grad = True

torch.stack([torch.stack([xx, xy]), torch.stack([yx, yy])])

tensor([[1., 2.],
        [3., 4.]], grad_fn=<StackBackward0>)